<a href="https://colab.research.google.com/github/SaiVardhan19/Movie_Recomendation/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv("/content/Netflix_Movies_and_TV_Shows.csv")

In [ ]:
df

,Title,Type,Genre,Release Year,Rating,Duration,Country
0,Title 1,TV Show,Comedy,1955,PG,3 Seasons,Japan
1,Title 2,TV Show,Horror,2020,G,3 Seasons,India
2,Title 3,TV Show,Action,1966,TV-PG,140 min,United States
3,Title 4,Movie,Thriller,2011,PG-13,3 Seasons,Canada
4,Title 5,TV Show,Romance,1959,TV-14,172 min,India
...,...,...,...,...,...,...,...
2995,Title 2996,Movie,Thriller,2007,TV-PG,75 min,Germany
2996,Title 2997,Movie,Drama,2019,G,2 Seasons,Germany
2997,Title 2998,TV Show,Action,1993,R,3 Seasons,Canada
2998,Title 2999,Movie,Drama,1966,PG-13,1 Seasons,Germany


In [ ]:
df['Combined Features'] = df['Type'] + ' ' + df['Genre'] + ' ' + df['Rating'] + ' ' + df['Country']

In [ ]:
# Step 2: Convert text features into TF-IDF matrix
# TF-IDF is used to determine the importance of the words

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Combined Features'])

In [ ]:
# Step 3: Compute cosine similarity matrix
# cosine similarity denotes the simiarity between the teo vectors
# cos x=(A.B)/(||A||*||B||)
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim

array([[1.        , 0.07759446, 0.2349197 , ..., 0.07883433, 0.11776609,
        0.19170648],
       [0.07759446, 1.        , 0.12981925, ..., 0.08586486, 0.        ,
        0.07097453],
       [0.2349197 , 0.12981925, 1.        , ..., 0.49247721, 0.09851416,
        0.68636227],
       ...,
       [0.07883433, 0.08586486, 0.49247721, ..., 1.        , 0.        ,
        0.07210862],
       [0.11776609, 0.        , 0.09851416, ..., 0.        , 1.        ,
        0.10771894],
       [0.19170648, 0.07097453, 0.68636227, ..., 0.07210862, 0.10771894,
        1.        ]])

In [ ]:
# Function to recommend based on user preferences
def recommend_by_preferences(preferences, num_recommendations=5):
    """
    preferences: A dictionary containing the user preferences from any category
    """
    # Filter out only the provided keys
    preference_parts = [str(preferences[key]) for key in ['Type', 'Genre', 'Rating', 'Country'] if key in preferences and preferences[key]]

    if not preference_parts:
        print("Error: No preferences provided.")
        return []

    # Create a user profile by concatenating the specified features
    user_profile = ' '.join(preference_parts)
    print(f"\nUser profile created from preferences: '{user_profile}'")

    # Transform the user profile using the TF-IDF vectorizer
    user_tfidf = tfidf_vectorizer.transform([user_profile])

    # Calculate cosine similarity between user preference and all items
    user_sim_scores = cosine_similarity(user_tfidf, tfidf_matrix).flatten()

    # Sort by similarity score
    movie_indices = user_sim_scores.argsort()[-num_recommendations:][::-1]

    return df.iloc[movie_indices][['Title', 'Type', 'Genre', 'Rating', 'Country']]


In [ ]:
# 1. Recommend based on only Genre
print("\n--- Recommendations based on Genre: 'Action' ---")
preferences = {'Genre': 'Action'}
recommendations = recommend_by_preferences(preferences, 5)
print(recommendations)


--- Recommendations based on Genre: 'Action' ---

User profile created from preferences: 'Action'
           Title     Type   Genre Rating Country
1164  Title 1165  TV Show  Action      R  Canada
2997  Title 2998  TV Show  Action      R  Canada
1819  Title 1820  TV Show  Action      G  Canada
254    Title 255  TV Show  Action      R  Canada
1695  Title 1696  TV Show  Action      G  Canada


In [ ]:
# 2. Recommend based on Genre and Type
print("\n--- Recommendations based on Genre: 'Comedy' and Type: 'Movie' ---")
preferences = {'Genre': 'Action', 'Type': 'TV Show'}
recommendations = recommend_by_preferences(preferences, 5)
print(recommendations)


--- Recommendations based on Genre: 'Comedy' and Type: 'Movie' ---

User profile created from preferences: 'TV Show Action'
           Title     Type   Genre Rating Country
1164  Title 1165  TV Show  Action      R  Canada
1819  Title 1820  TV Show  Action      G  Canada
2997  Title 2998  TV Show  Action      R  Canada
254    Title 255  TV Show  Action      R  Canada
1695  Title 1696  TV Show  Action      G  Canada


In [ ]:
# 3. Recommend based on Genre, Type, and Rating
print("\n--- Recommendations based on Genre: 'Comedy', Type: 'Movie', and Rating: 'R' ---")
preferences = {'Genre': 'Comedy', 'Type': 'Movie', 'Rating': 'R'}
recommendations = recommend_by_preferences(preferences, 5)
print(recommendations)


--- Recommendations based on Genre: 'Comedy', Type: 'Movie', and Rating: 'R' ---

User profile created from preferences: 'Movie Comedy R'
           Title   Type   Genre Rating  Country
554    Title 555  Movie  Comedy      R   Canada
2764  Title 2765  Movie  Comedy      R   Canada
2876  Title 2877  Movie  Comedy      R   Canada
888    Title 889  Movie  Comedy      R   Canada
1208  Title 1209  Movie  Comedy      R  Germany


In [ ]:
# 4. Recommend based on all features (Genre, Type, Rating, and Country)
print("\n--- Recommendations based on Genre: 'Action', Type: 'Movie', Rating: 'PG-13', and Country: 'USA' ---")
preferences = {'Genre': 'Action', 'Type': 'Movie', 'Rating': 'PG-13', 'Country': 'USA'}
recommendations = recommend_by_preferences(preferences, 5)
print(recommendations)


--- Recommendations based on Genre: 'Action', Type: 'Movie', Rating: 'PG-13', and Country: 'USA' ---

User profile created from preferences: 'Movie Action PG-13 USA'
           Title   Type   Genre Rating  Country
666    Title 667  Movie  Action  PG-13   Canada
1638  Title 1639  Movie  Action  PG-13   Canada
898    Title 899  Movie  Action  PG-13   Canada
2656  Title 2657  Movie  Action  PG-13   Canada
592    Title 593  Movie  Action  PG-13  Germany
